In [15]:
from langchain.utilities.wikipedia import WikipediaAPIWrapper
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import openai as client
from bs4 import BeautifulSoup
import requests
def DuckDuckSearchTool(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    query = inputs["query"]
    results = ddg.run(query)
    # Assuming the results are a JSON object and the first result's URL is accessible via ['results'][0]['url']
    if results and 'results' in results and len(results['results']) > 0:
        url = results['results'][0]['url']  # Access the URL of the first result
        print("url:",url)
    print("results:",results)
    return results


def WikiSearchTool(inputs):
    wiki = WikipediaAPIWrapper()
    query = inputs["query"]
    results = wiki.run(query)
    if results and 'results' in results and len(results['results']) > 0:
        url = results['results'][0]['url']  # Access the URL of the first result
        print("url:",url)
    print("results:",results)    
    return results

def SaveToTextFileTool(inputs):
    try:
        file_path = f"MyResearch.txt"
        if "url" in inputs.keys():
            url = inputs["url"]
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            response.raise_for_status()  # Raises an error for bad responses
            title_text = soup.find('title').text if soup.find('title') else 'Title tag not found'
        else:
            text = inputs["text"]
            with open(file_path, 'w') as f:
                f.write(text)
            return "Text saved to file."
    except requests.RequestException as e:
        with open(file_path, 'w') as f:
            f.write(text)     
        return f"Failed to load URL: {str(e)}"
    except ValueError:
        with open(file_path, 'w') as f:
            f.write(text)     
        return "URL did not return a JSON response."       


     
functions = [
    {
        "type": "function",
        "function": {
            "name": "DuckDuckSearchTool",
            "description": "Use this tool to search for the provided query and provide one url from the searching.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query that user wants to search",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "WikiSearchTool",
            "description": "Use this tool to search for the provided query and provide one url from the searching.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query that user wants to search",
                    },
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "SaveToTextFileTool",
            "description": "Use this tool to save the text file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text that user wants to save to the text file.",
                    },
                    "url": {
                        "type": "string",
                        "description": "The url that user wants to enter and extract text to save as the text file.",
                    },
                },
                "required": ["text"],
            },
        },
    },
]

assistant = client.beta.assistants.create(
    name="Research Assistant",
    instructions="""You help users do research from the web.
    Search the query using DuckDuckSearchTool and extract the content from the link.
    Search the query using WikiSearchTool and extract the content from the link.
    if there is link from the content, enter the link and extract the content.
    if there is no link just save text content from the research.
    Save the content as text file.""",
    model="gpt-4-1106-preview",
    tools=functions,
)        

In [19]:
abb={"role": "assistant", "content": "It seems you might want to initiate some sort of activity, but your message is quite vague. Could you please provide more details about what you would like to do? If you have a specific question, topic, or task in mind, please let me know so I can assist you accordingly. If you're looking to use one of the tools available, please specify which tool you'd like to use and the necessary details for your request."}
abb.role


AttributeError: 'dict' object has no attribute 'role'

In [13]:
DuckDuckSearchTool({"query":"GenZ"})

results: Learn the latest definitions and birth years for different generational groups, including Gen Z and Gen Alpha. Find out how these generations are shaped by historical, economic, and technological factors. Generation Z (often shortened to Gen Z), colloquially known as Zoomers, is the demographic cohort succeeding Millennials and preceding Generation Alpha.Researchers and popular media use the mid-to-late 1990s as starting birth years and the early 2010s as ending birth years. Most members of Generation Z are the children of Generation X or older Millennials. Key insights in 4 charts. Gen Z is the generation born in the 15-year span from 1997 to 2012. In 2024, they will turn between 12 and 27 years old. This is the newest generation to gain influence in the world, from joining the workforce to being old enough to vote. The oldest Gen Zers may have already voted in 2020 and possibly the 2016 elections. Learn who are the Gen Zers, the most racially and ethnically diverse generatio

'Learn the latest definitions and birth years for different generational groups, including Gen Z and Gen Alpha. Find out how these generations are shaped by historical, economic, and technological factors. Generation Z (often shortened to Gen Z), colloquially known as Zoomers, is the demographic cohort succeeding Millennials and preceding Generation Alpha.Researchers and popular media use the mid-to-late 1990s as starting birth years and the early 2010s as ending birth years. Most members of Generation Z are the children of Generation X or older Millennials. Key insights in 4 charts. Gen Z is the generation born in the 15-year span from 1997 to 2012. In 2024, they will turn between 12 and 27 years old. This is the newest generation to gain influence in the world, from joining the workforce to being old enough to vote. The oldest Gen Zers may have already voted in 2020 and possibly the 2016 elections. Learn who are the Gen Zers, the most racially and ethnically diverse generation of Ame

In [16]:
WikiSearchTool({"query": "GenZ"})

'Page: Generation Z\nSummary: Generation Z (often shortened to Gen Z), colloquially known as Zoomers, is the demographic cohort succeeding Millennials and preceding Generation Alpha. Researchers and popular media use the mid-to-late 1990s as starting birth years and the early 2010s as ending birth years. Most members of Generation Z are the children of Generation X or older Millennials.As the first social generation to have grown up with access to the Internet and portable digital technology from a young age, members of Generation Z, even if not necessarily digitally literate, have been dubbed "digital natives". Moreover, the negative effects of screen time are most pronounced in adolescents, as compared to younger children. Compared to previous generations, members of Generation Z tend to live more slowly than their predecessors when they were their age, have lower rates of teenage pregnancies, and consume alcohol (but not necessarily other psychoactive drugs) less often. Generation Z

In [17]:
SaveToTextFileTool({"text":"Generation Z (often shortened to Gen Z), colloquially known as Zoomers, is the demographic cohort succeeding Millennials and preceding Generation Alpha. Researchers and popular media use the mid-to-late 1990s as starting birth years and the early 2010s as ending birth years. Most members of Generation Z are the children of Generation X or older Millennials.\n\nAs the first social generation to have grown up with access to the Internet and portable digital technology from a young age, members of Generation Z, even if not necessarily digitally literate, have been dubbed \"digital natives\". Moreover, the negative effects of screen time are most pronounced in adolescents, as compared to younger children. Compared to previous generations, members of Generation Z tend to live more slowly than their predecessors when they were their age, have lower rates of teenage pregnancies, and consume alcohol (but not necessarily other psychoactive drugs) less often. Generation Z teenagers are more concerned than older generations with academic performance and job prospects, and are better at delaying gratification than their counterparts from the 1960s, despite concerns to the contrary.\n\nSexting among adolescents has grown in prevalence; the consequences of this remain poorly understood. Youth subcultures have not disappeared, but they have been quieter. Nostalgia is a major theme of youth culture in the 2010s and 2020s.\n\nGlobally, there is evidence that the average age of pubertal onset among girls has decreased considerably compared to the 20th century, with implications for their welfare and their future. Furthermore, the prevalence of allergies among adolescents and young adults in Generation Z is greater than the general population; there is greater awareness and diagnosis of mental health conditions, and sleep deprivation is more frequently reported. In many countries, Gen Z youth are more likely to be diagnosed with intellectual disabilities and psychiatric disorders than older generations.\n\nAround the world, members of Generation Z are spending more time on electronic devices and less time reading books than before, with implications for their attention spans, vocabulary, academic performance, and future economic contributions. In Asia, educators in the 2000s and 2010s typically sought out and nourished top students; in Western Europe and the United States, the emphasis was on poor performers. Furthermore, East Asian and Singaporean students consistently earned the top spots in international standardized tests in the 2010s.\n\nThe Gen-Z Consortium is a trade group of technology vendors involved in designing CPUs, random access memory, servers, storage, and accelerators. The goal was to design an open and royalty-free \"memory-semantic\" bus protocol, which is not limited by the memory controller of a CPU, to be used in either a switched fabric or a point-to-point device link on a standard connector. In November 2021, the GenZ Consortium voted to transfer all its specifications and intellectual property to the CXL Consortium."})

'Text saved to file.'